# Peoject WIA
#### Matthieu Ré and Yaohui WANG
### Part1 Treatment of the dataset
### Variables:
1. final_sentence: the pure sentences in the dataset, no punctuations, no numbers, low letters
2. final_couple: the nouns marks in the sentence
3. list_index: the locals of the nous in the sentences

In [2]:
f = open("SemEval2010_task8_training/TRAIN_FILE.TXT")

# read the train data line by line 
# store in a list
def readinFile(f):
    filelist = []
    line = f.readline()
    filelist.append(line)
    while line:
        line = f.readline()
        filelist.append(line)
    return filelist

filelist = readinFile(f)

In [3]:
# using 3 lists to store 3 elements
# list_sentence: each sentence in the list
# list_class: each class in the list
# list comment: each comment in the list
def separateFilelist(l):
    list_sentence = []
    list_class = []
    list_comment = []
    for i in range(len(l)):
        if (i + 1) % 4 == 1:
            list_sentence.append(l[i])
        if (i + 1) % 4 == 2:
            list_class.append(l[i])
        if (i + 1) % 4 == 3:
            list_comment.append(l[i])
    # to remove the '' in the end, no use
    list_sentence.remove('')
    return list_sentence,list_class,list_comment

list_sentence,list_class,list_comment = separateFilelist(filelist)

# here we delete the sentences which give us noises
# index: 212,2739,4218,4611,4783,6372

list_sentence.remove(list_sentence[212])
list_sentence.remove(list_sentence[2738])
list_sentence.remove(list_sentence[4216])
list_sentence.remove(list_sentence[4608])
list_sentence.remove(list_sentence[4779])
list_sentence.remove(list_sentence[6367])

list_class.remove(list_class[212])
list_class.remove(list_class[2738])
list_class.remove(list_class[4216])
list_class.remove(list_class[4608])
list_class.remove(list_class[4779])
list_class.remove(list_class[6367])

list_comment.remove(list_comment[212])
list_comment.remove(list_comment[2738])
list_comment.remove(list_comment[4216])
list_comment.remove(list_comment[4608])
list_comment.remove(list_comment[4779])
list_comment.remove(list_comment[6367])

In [4]:
def sizeoftrain(list_sentence,list_class,list_comment):
    print("The length of the sentences is: ",len(list_sentence))
    print("The length of the classes is: ",len(list_class))
    print("The length of the comments is: ",len(list_comment))
    
sizeoftrain(list_sentence,list_class,list_comment)

The length of the sentences is:  7994
The length of the classes is:  7994
The length of the comments is:  7994


In [5]:
from bs4 import BeautifulSoup
def getNouns(list_sentence):
    l_noun = []
    for sentence in list_sentence:
        flag = 0
        l1 = ''
        l2 = ''
        couple_noun = []
        for i in range(len(sentence)):
            if (sentence[i] == '<') and (sentence[i+1] == 'e') and (sentence[i+2] == '1'):
                flag = 1
            if (sentence[i] == '>') and (sentence[i-1] == '1') and (sentence[i-2] == 'e') and (sentence[i-3] == '/'):
                flag = 0
            if (sentence[i] == '<') and (sentence[i+1] == 'e') and (sentence[i+2] == '2'):
                flag = 2
            if (sentence[i] == '>') and (sentence[i-1] == '2') and (sentence[i-2] == 'e') and (sentence[i-3] == '/'):
                flag = 0

            if flag == 1:
                l1 += sentence[i]
            if flag == 2:
                l2 += sentence[i]
        noun1_b = BeautifulSoup(l1)
        noun2_b = BeautifulSoup(l2)
        noun1 = noun1_b.get_text()
        noun2 = noun2_b.get_text()
        couple_noun.append(noun1)
        couple_noun.append(noun2)
        l_noun.append(couple_noun)
    return l_noun

l_noun = getNouns(list_sentence)

/home/sirius/anaconda3/envs/theano/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/sirius/anaconda3/envs/theano/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [6]:
# replace the space in the marked nous into #, and replace these nous in the sentences
# example: WANG Yaohui -> WANG#Yaohui

import nltk.data
import re

def replace_the_space(l_noun):
    l_new_noun = []
    for couple in l_noun:
        c = (couple[0].replace(' ','#'),couple[1].replace(' ','#'))
        l_new_noun.append(c)
    return l_new_noun

def replace_original_noun(list_sentence,l_new_noun):
    l_new_sentence = []
    for sentence,couple in zip(list_sentence,l_new_noun):
        left = couple[0].replace('#',' ')
        right = couple[1].replace('#',' ')
        sentence = re.sub("<e1>" + left + "</e1>",couple[0],sentence)
        sentence = re.sub("<e2>" + right + "</e2>",couple[1],sentence)
        l_new_sentence.append(sentence)
    return l_new_sentence

def get_the_new_sentence(list_sentence,l_noun):
    final_sentence = []
    
    l_new_noun = replace_the_space(l_noun)
    l_new_sentence = replace_original_noun(list_sentence,l_new_noun)
    
    for l in l_new_sentence:
        sentence_text = re.sub("[^a-zA-Z#-]"," ",l)
        words = sentence_text.lower().split()
        final_sentence.append(words)
    return final_sentence

def get_the_final_couple(l_noun):
    new_couple = replace_the_space(l_noun)
    final_couple = []
    for couple in new_couple:
        c = (couple[0].lower(),couple[1].lower())
        final_couple.append(c)
    return final_couple

# final sentences we need
final_sentence = get_the_new_sentence(list_sentence,l_noun)
# final marked nouns we need
final_couple = get_the_final_couple(l_noun)

In [7]:
def get_location(final_sentence,final_couple):
    list_index = []
    for couple,sentence in zip(final_couple,final_sentence):
        #print(sentence)
        #print(couple[0] + couple[1])
        noun_couple = (sentence.index(couple[0]),sentence.index(couple[1]))
        list_index.append(noun_couple)
    return list_index

# location of marked nouns in the sentences
list_index = get_location(final_sentence, final_couple)

## word2vec

In [163]:
# apply word2vec on our final sentences list
from gensim import models
import logging
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s', level = logging.INFO)

num_features = 50
min_word_count = 1
num_workers = 8
context = 10
downsampling = 0.0001
print("Training model...")

model = models.word2vec.Word2Vec(final_sentence, workers = num_workers, size = num_features, min_count = min_word_count, window = context, sample = downsampling)

2017-01-09 01:09:40,533: WARNING: consider setting layer size to a multiple of 4 for greater performance
2017-01-09 01:09:40,535: INFO: collecting all words and their counts
2017-01-09 01:09:40,536: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-01-09 01:09:40,574: INFO: collected 19818 word types from a corpus of 136793 raw words and 7994 sentences
2017-01-09 01:09:40,575: INFO: Loading a fresh vocabulary
2017-01-09 01:09:40,692: INFO: min_count=1 retains 19818 unique words (100% of original 19818, drops 0)
2017-01-09 01:09:40,693: INFO: min_count=1 leaves 136793 word corpus (100% of original 136793, drops 0)


Training model...


2017-01-09 01:09:40,762: INFO: deleting the raw counts dictionary of 19818 items
2017-01-09 01:09:40,764: INFO: sample=0.0001 downsamples 330 most-common words
2017-01-09 01:09:40,765: INFO: downsampling leaves estimated 79036 word corpus (57.8% of prior 136793)
2017-01-09 01:09:40,766: INFO: estimated required memory for 19818 words and 50 dimensions: 17836200 bytes
2017-01-09 01:09:40,830: INFO: resetting layer weights
2017-01-09 01:09:41,100: INFO: training model with 8 workers on 19818 vocabulary and 50 features, using sg=0 hs=0 sample=0.0001 negative=5 window=10
2017-01-09 01:09:41,101: INFO: expecting 7994 sentences, matching count from corpus used for vocabulary survey
2017-01-09 01:09:41,109: WARNING: direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab
2017-01-09 01:09:41,109: WARNING: direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab
2017-01-09 01:09:41,110: WARNING: direct access to 

In [164]:
model.init_sims(replace = True)

model_name = "wia"
model.save(model_name)

2017-01-09 01:09:45,381: INFO: precomputing L2-norms of word weight vectors
2017-01-09 01:09:45,517: INFO: saving Word2Vec object under wia, separately None
2017-01-09 01:09:45,518: WARNING: direct access to syn0norm will not be supported in future gensim releases, please use model.wv.syn0norm
2017-01-09 01:09:45,519: WARNING: direct access to syn0norm will not be supported in future gensim releases, please use model.wv.syn0norm
2017-01-09 01:09:45,520: WARNING: direct access to syn0norm will not be supported in future gensim releases, please use model.wv.syn0norm
2017-01-09 01:09:45,520: INFO: not storing attribute cum_table
2017-01-09 01:09:45,521: INFO: not storing attribute syn0norm
2017-01-09 01:09:45,622: WARNING: direct access to syn0norm will not be supported in future gensim releases, please use model.wv.syn0norm
2017-01-09 01:09:45,623: INFO: saved wia


In [165]:
model.doesnt_match("man waman child kitchen".split())

'child'

### Lexical Level features

In [166]:
import numpy as np

# L1 L2 L3 L4
def get_lexical_feature(final_couple,list_sentence,final_sentence):
    lexical_feature = []
    for couple,index,sentence in zip(final_couple,list_index,final_sentence):

        noun1_v = model[couple[0]]
        noun2_v = model[couple[1]]
    
        if index[0] == 0:
            noun3_v = model[sentence[index[0]+1]] / 2 
        if index[0] == (len(sentence)-1):
            noun3_v = model[sentence[index[0]-1]] / 2
        else:
            noun3_v = (model[sentence[index[0]-1]] + model[sentence[index[0]+1]]) / 2
        
        if index[1] == 0:
            noun4_v = model[sentence[index[1]+1]] / 2 
        if index[1] == (len(sentence)-1):
            noun4_v = model[sentence[index[1]-1]] / 2
        else:
            noun4_v = (model[sentence[index[1]-1]] + model[sentence[index[1]+1]]) / 2
    
        word_feature = np.concatenate((noun1_v,noun2_v,noun3_v,noun4_v))
        lexical_feature.append(word_feature)
    return np.array(lexical_feature)

lexical_feature = get_lexical_feature(final_couple,list_sentence,final_sentence)

In [167]:
lexical_feature.shape

(7994, 200)

### Sentence Level Feature
#### Part 1
get the [WF, PF]

In [168]:
# define the beginning and the end 
xs = np.zeros(50)
xe = np.zeros(50)

In [181]:
# get the [WF, PF]
def get_the_sentence_feature(final_sentence,list_index):
    window = 3
    list_sentence_feature = []
    for sentence,position in zip(final_sentence,list_index):
        wf_l = []
        for index in range(len(sentence)):
            if index == 0:
                wf = np.concatenate((xs, model[sentence[index]], model[sentence[index+1]], [index-position[0]], [index-position[1]]))
            if index == (len(sentence)-1):
                wf = np.concatenate((model[sentence[index-1]], model[sentence[index]], xe, [index-position[0]], [index-position[1]]))
            else:
                wf = np.concatenate((model[sentence[index-1]], model[sentence[index]], model[sentence[index+1]], [index-position[0]], [index-position[1]]))
            wf_l.append(wf)
        list_sentence_feature.append(wf_l)
    return list_sentence_feature

list_sentence_feature = get_the_sentence_feature(final_sentence,list_index)

In [229]:
len(list_sentence_feature[0])

16

#### Part 2
##### Training the model CNN and get the sentence level feature
Layer 1

Activate Function: Max

In [206]:
def max_f(z):
    res_l1 = z.max(axis=1)
    return res_l1

def layer_1(W1,x):
    a = np.dot(W1,np.array(x).transpose())
    m = max_f(a)
    return m

Theano version

In [218]:
'''
import theano
import theano.tensor as T
from theano.ifelse import ifelse
import numpy as np

x1 = T.matrix('x1')
w1 = theano.shared(np.ones([200,152]))

z1 = T.dot(w1,x1.transpose())
a1 = z1.max(axis=1)

layer1 = theano.function([x1],a1)

inputs = list_sentence_feature

list_out = []
for i in range(len(inputs)):
    t = inputs[i]
    out = layer1(t)
    list_out.append(out)

list_out = np.array(list_out)
'''

Layer 2

Activate Function: tanh

In [207]:
def tanh_f(z):
    res_l2 = np.tanh(z)
    return res_l2

def layer_2(W2,x):
    a = np.dot(W2,x)
    m = np.tanh(a)
    return m

In [208]:
# n1 = 200
# n2 = 100
def cnn_sentence_feature(n1,n2,list_sentence_feature):
    n0 = 152 # the length of each word feature vecteur
    W1 = np.ones((n1,n0))
    l1_output = []
    
    W2 = np.ones((n2,n1))
    l2_output = []
    
    for x in list_sentence_feature:
        # first layer
        m1 = layer_1(W1,x)
        l1_output.append(m1)
        
        # second layer
        m2 = layer_2(W2,m1)
        l2_output.append(m2)
    return l1_output,l2_output

In [209]:
l1_output, l2_output = cnn_sentence_feature(200,100,list_sentence_feature)

Layer output 

Activate Function: softmax

In [214]:
# lexical level feature + sentence level feature
list_feature_final = []
for wf,sf in zip(lexical_feature,l2_output):
    feature_final = np.concatenate((sf,wf))
    list_feature_final.append(feature_final)

In [216]:
def softmax(z):
    p = np.max(z,0)
    return np.exp(z-p) / np.sum(np.exp(z-p), axis = 0)

In [217]:
def layer_3(W3,x):
    a = np.dot(W3,x)
    m = softmax(a)
    return m